### I now have tables of precip, snowfall, snowpack, weather, and avy dangers. now to finally put it all together, hopefully!

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## There are many weather stations across the Steven Pass area. 
### I will choose the tye-mill station for it is the closest to the Brooks elevation for the precipitaion data from the Brooks station
### Therefore, I will be predicting the near treeline avalanche danger

In [2]:
tyemill_2019 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2019.csv")
tyemill_2018 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2018.csv")
tyemill_2017 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2017.csv")
tyemill_2016 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2016.csv")
tyemill_2015 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2015.csv")
tyemill_2014 = pd.read_csv("csv_files/StevensPass-TyeMill_5180_feet_2014.csv")

#sanity checks below to ensure I am not losing any rows
tyemill_years = [tyemill_2019,tyemill_2018,tyemill_2017,tyemill_2016,tyemill_2015,tyemill_2014,]
result = 0
for year in tyemill_years:
    print(year.shape)
    result += year.shape[0]
    
print(result)

(3200, 7)
(8762, 7)
(8760, 7)
(8789, 7)
(8712, 7)
(346, 7)
38569


In [3]:
tyemill = pd.concat(tyemill_years,axis=0)
tyemill = tyemill.reset_index(drop=True)
tyemill.shape

(38569, 7)

In [4]:
tyemill.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.)
0,2019-05-14 07:00,12.80,37.21,0.000,4.660,20.34,250.3
1,2019-05-14 06:00,12.84,44.51,0.000,4.046,10.11,278.6
2,2019-05-14 05:00,12.94,43.41,0.723,7.701,15.02,272.7
3,2019-05-14 04:00,13.57,43.25,4.340,10.050,18.87,271.9
4,2019-05-14 03:00,12.64,42.71,3.288,9.440,17.34,275.2


In [8]:
display(tyemill.columns)
tyemill.info()
#don't worry about cleaning data yet, lets first merge to reduce doing it twice!

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Temperature (deg F)',
       'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
       'Wind Speed Maximum (mph)', 'Wind Direction (deg.)'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38569 entries, 0 to 38568
Data columns (total 7 columns):
Date/Time (PST)             38569 non-null object
Battery Voltage (v)         38560 non-null float64
Temperature (deg F)         38559 non-null float64
Wind Speed Minimum (mph)    38559 non-null float64
Wind Speed Average (mph)    38559 non-null float64
Wind Speed Maximum (mph)    38559 non-null float64
Wind Direction (deg.)       36887 non-null float64
dtypes: float64(6), object(1)
memory usage: 2.1+ MB


## I want to extract the date without the time of day so i can match this to my avalanche table

In [9]:
type(tyemill['Date/Time (PST)'][1])

str

In [10]:
tyemill['date'] = tyemill['Date/Time (PST)'].apply(lambda x: x.split(' ')[0])


In [11]:
tyemill.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date
0,2019-05-14 07:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14
1,2019-05-14 06:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14
2,2019-05-14 05:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14
3,2019-05-14 04:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14
4,2019-05-14 03:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14


In [12]:
tyemill.shape

(38569, 8)

In [13]:
tyemill['Date/Time (PST)'] = pd.to_datetime(tyemill['Date/Time (PST)'])
tyemill['date'] = pd.to_datetime(tyemill['date'])

In [14]:
type(tyemill['Date/Time (PST)'][1]),type(tyemill['date'][1])

(pandas._libs.tslibs.timestamps.Timestamp,
 pandas._libs.tslibs.timestamps.Timestamp)

In [15]:
tyemill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38569 entries, 0 to 38568
Data columns (total 8 columns):
Date/Time (PST)             38569 non-null datetime64[ns]
Battery Voltage (v)         38560 non-null float64
Temperature (deg F)         38559 non-null float64
Wind Speed Minimum (mph)    38559 non-null float64
Wind Speed Average (mph)    38559 non-null float64
Wind Speed Maximum (mph)    38559 non-null float64
Wind Direction (deg.)       36887 non-null float64
date                        38569 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(6)
memory usage: 2.4 MB


In [16]:
# pd.to_datetime(skyline_df['Date/Time (PST)'], '%Y-%m-%d %H:%m') # this could work but it recognized it anyhow

In [17]:
tyemill.head(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date
0,2019-05-14 07:00:00,12.8,37.21,0.0,4.66,20.34,250.3,2019-05-14


In [18]:
tyemill.isna().sum() #sanity check, we are still good

Date/Time (PST)                0
Battery Voltage (v)            9
Temperature (deg F)           10
Wind Speed Minimum (mph)      10
Wind Speed Average (mph)      10
Wind Speed Maximum (mph)      10
Wind Direction (deg.)       1682
date                           0
dtype: int64

In [23]:
tyemill.describe()
## Looks like there are negative values across the data, but again lets not worry about it until afte the merge

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.)
count,38560.000000,38559.000000,38559.000000,38559.000000,38559.000000,36887.000000
mean,12.612753,36.991968,2.047324,6.683592,12.877702,201.488927
std,1.718576,59.728811,3.881819,5.345024,7.911821,94.258006
min,0.000000,-7999.000000,-7.743000,0.000000,0.000000,0.000000
25%,12.600000,26.800000,0.000000,3.181000,7.672000,93.300000
50%,12.670000,35.210000,0.395000,5.302000,10.810000,258.200000
75%,12.750000,46.440000,2.959000,8.830000,16.660000,272.600000
max,45.980000,87.900000,53.250000,84.900000,71.790000,359.900000


In [25]:
brooks_precip_2019 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2019.csv')
brooks_precip_2018 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2018.csv')
brooks_precip_2017 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2017.csv')
brooks_precip_2016 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2016.csv')
brooks_precip_2015 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2015.csv')
brooks_precip_2014 = pd.read_csv('csv_files/StevensPass-BrooksPrecipitation_4800_feet_2014.csv')

brooks_years = [brooks_precip_2019,brooks_precip_2018,brooks_precip_2017,
                 brooks_precip_2016,brooks_precip_2015,brooks_precip_2014]

result = 0
for year in brooks_years:
    print(year.shape)
    result += year.shape[0]
    
print(result)

(3843, 7)
(8762, 7)
(8761, 7)
(8784, 7)
(863, 7)
(0, 7)
31013


In [26]:
brooks_precip_2014.head(10)

,Date/Time (PST),Battery Voltage (v),"Precipitation ("")","24 Hour Snow ("")","Total Snow Depth ("")",Temperature (deg F),Relative Humidity (%)


In [26]:
brooks_precip = pd.concat(brooks_years,axis=0)
brooks_precip = brooks_precip.reset_index(drop=True)

In [27]:
brooks_precip.shape

(31013, 7)

In [28]:
brooks_precip.describe()

,Battery Voltage (v),"Precipitation ("")","24 Hour Snow ("")","Total Snow Depth ("")",Temperature (deg F),Relative Humidity (%)
count,31010.000000,31010.000000,31010.000000,31010.000000,1142.000000,1142.000000
mean,13.461448,0.008431,-166.949569,53.042241,43.338555,74.241953
std,0.458908,0.122036,1105.334526,50.645324,8.735684,21.295240
min,0.000000,0.000000,-7999.000000,-164.000000,22.810000,20.510000
25%,13.100000,0.000000,0.207250,3.490500,37.177500,57.862500
50%,13.460000,0.000000,1.065000,46.985000,42.300000,76.395000
75%,13.830000,0.000000,53.917500,93.900000,48.612500,94.375000
max,15.100000,21.090000,529.500000,281.100000,67.090000,100.000000


In [29]:
brooks_precip.columns #need to get rid of the quotes for inches, rename columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Precipitation (")',
       '24 Hour Snow (")', 'Total Snow Depth (")', 'Temperature (deg F)',
       'Relative Humidity (%)'],
      dtype='object')

In [35]:
brooks_precip.columns=['Date/Time (PST)', 'Battery Voltage (v)', 'Precipitation (in)',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'Temperature (deg F)',
       'Relative Humidity (%)']
brooks_precip.tail(1)

,Date/Time (PST),Battery Voltage (v),Precipitation (in),24 Hour Snow (in),Total Snow Depth (in),Temperature (deg F),Relative Humidity (%)
31012,2015-11-26 01:00:00,13.2,0.0,1.591,0.0,NaN,NaN


In [31]:
brooks_precip['Date/Time (PST)'] = pd.to_datetime(brooks_precip['Date/Time (PST)'])

In [32]:
brooks_precip['Date/Time (PST)'][0], tyemill['Date/Time (PST)'][0]

(Timestamp('2019-06-10 06:00:00'), Timestamp('2019-05-14 07:00:00'))

In [34]:
brooks_precip.head(1)

,Date/Time (PST),Battery Voltage (v),Precipitation (in),24 Hour Snow (in),Total Snow Depth (in),Temperature (deg F),Relative Humidity (%)
0,2019-06-10 06:00:00,9.07,0.0,65.14,7.095,50.43,65.03


## I only want the 24 snow, snow depth, and rel humidity, and date

In [42]:
brooks_snow =  brooks_precip[['24 Hour Snow (in)','Total Snow Depth (in)','Relative Humidity (%)','Date/Time (PST)']]

In [46]:
brooks_snow = brooks_snow.drop_duplicates()

In [47]:
merged_df =pd.merge(tyemill,brooks_snow,how='left',on='Date/Time (PST)')

In [49]:
merged_df.shape,tyemill.shape,brooks_snow.shape

((38569, 11), (38569, 8), (31012, 4))

In [50]:
merged_df.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%)
0,2019-05-14 07:00:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14,11.73,40.14,89.30
1,2019-05-14 06:00:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14,13.34,39.53,74.99
2,2019-05-14 05:00:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14,13.34,39.27,69.16
3,2019-05-14 04:00:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14,14.68,40.21,58.96
4,2019-05-14 03:00:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14,13.95,40.47,62.37


In [51]:
merged_df.tail(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%)
38568,2014-12-17 13:00:00,0.0,12.89,25.3,0.008,0.81,125.4,2014-12-17,NaN,NaN,NaN


In [52]:
# Now to add in the avalanche stuff
#also keep in mind, the date from the forecast danger is a prediction on tomorrow,
#so to match it I must make a new column that has the correct day

In [53]:
stevens_avalanches = pd.read_json('stevens_forecast_danger.json',lines=True)
stevens_avalanches.head()

,area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow
0,Stevens Pass,Moderate,Moderate,Moderate,"December 31, 2018"
1,Stevens Pass,Moderate,Moderate,Moderate,"December 31, 2014"
2,Stevens Pass,Considerable,Moderate,Considerable,"December 30, 2014"
3,Stevens Pass,Considerable,Moderate,Moderate,"December 29, 2014"
4,Stevens Pass,Considerable,Moderate,Moderate,"December 29, 2014"


In [54]:
stevens_avalanches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 5 columns):
area                     967 non-null object
danger_above_treeline    967 non-null object
danger_below_treeline    967 non-null object
danger_near_treeline     967 non-null object
date_tomorrow            967 non-null object
dtypes: object(5)
memory usage: 37.9+ KB


In [55]:
# d = datetime.today() - timedelta(days=days_to_subtract)

In [56]:
stevens_avalanches['date'] = (pd.to_datetime(stevens_avalanches['date_tomorrow'])-timedelta(days=1))

In [57]:
stevens_avalanches.head()

,area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,date
0,Stevens Pass,Moderate,Moderate,Moderate,"December 31, 2018",2018-12-30
1,Stevens Pass,Moderate,Moderate,Moderate,"December 31, 2014",2014-12-30
2,Stevens Pass,Considerable,Moderate,Considerable,"December 30, 2014",2014-12-29
3,Stevens Pass,Considerable,Moderate,Moderate,"December 29, 2014",2014-12-28
4,Stevens Pass,Considerable,Moderate,Moderate,"December 29, 2014",2014-12-28


In [58]:
second_merge =pd.merge(merged_df,stevens_avalanches,how='left',on='date')
second_merge.head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow
0,2019-05-14 07:00:00,12.80,37.21,0.000,4.660,20.34,250.3,2019-05-14,11.73,40.14,89.30,NaN,NaN,NaN,NaN,NaN
1,2019-05-14 06:00:00,12.84,44.51,0.000,4.046,10.11,278.6,2019-05-14,13.34,39.53,74.99,NaN,NaN,NaN,NaN,NaN
2,2019-05-14 05:00:00,12.94,43.41,0.723,7.701,15.02,272.7,2019-05-14,13.34,39.27,69.16,NaN,NaN,NaN,NaN,NaN
3,2019-05-14 04:00:00,13.57,43.25,4.340,10.050,18.87,271.9,2019-05-14,14.68,40.21,58.96,NaN,NaN,NaN,NaN,NaN
4,2019-05-14 03:00:00,12.64,42.71,3.288,9.440,17.34,275.2,2019-05-14,13.95,40.47,62.37,NaN,NaN,NaN,NaN,NaN


In [59]:
# I only want the ski season months where nwac gives forecasts, Dec-April 
# in datetime january = 1 and december = 12
# so I want months 12,1,2,3,4

In [60]:
second_merge['month'] = pd.DatetimeIndex(second_merge['date']).month

In [61]:
type(second_merge['month'][0]) #it is an int

ski_months = [12,1,2,3,4]

In [88]:
ski_df = second_merge.loc[second_merge['month'].isin(ski_months)]

In [89]:
ski_df['month'].value_counts()

1     4730
3     4488
12    4306
2     3961
4     3818
Name: month, dtype: int64

In [90]:
ski_df.head(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,month
320,2019-04-30 23:00:00,12.8,33.07,5.962,9.23,13.33,267.0,2019-04-30,54.89,74.06,52.35,NaN,NaN,NaN,NaN,NaN,4


In [91]:
ski_df['danger_above_treeline'][3219]

'Moderate'

In [92]:
ski_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21303 entries, 320 to 42119
Data columns (total 17 columns):
Date/Time (PST)             21303 non-null datetime64[ns]
Battery Voltage (v)         21296 non-null float64
Temperature (deg F)         21295 non-null float64
Wind Speed Minimum (mph)    21295 non-null float64
Wind Speed Average (mph)    21295 non-null float64
Wind Speed Maximum (mph)    21295 non-null float64
Wind Direction (deg.)       19429 non-null float64
date                        21303 non-null datetime64[ns]
24 Hour Snow (in)           17685 non-null float64
Total Snow Depth (in)       17685 non-null float64
Relative Humidity (%)       178 non-null float64
area                        19682 non-null object
danger_above_treeline       19682 non-null object
danger_below_treeline       19682 non-null object
danger_near_treeline        19682 non-null object
date_tomorrow               19682 non-null object
month                       21303 non-null int64
dtypes: datetime6

## I need to convert the string avalanche levels to numbers, lets make a dictionary and then map it to the columns

In [93]:
type(ski_df['danger_above_treeline'][2456])

str

In [94]:
def convert_avy_str_to_num(word):
    if word == 'Low':
        return 1
    elif word == 'Moderate':
        return 2
    elif word == 'Considerable':
        return 3
    elif word == 'High':
        return 4
    elif word == 'Extreme':
        return 5

In [95]:
ski_df.columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Temperature (deg F)',
       'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
       'Wind Speed Maximum (mph)', 'Wind Direction (deg.)', 'date',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'Relative Humidity (%)',
       'area', 'danger_above_treeline', 'danger_below_treeline',
       'danger_near_treeline', 'date_tomorrow', 'month'],
      dtype='object')

In [97]:
ski_df[['danger_above_treeline', 'danger_below_treeline','danger_near_treeline']] = ski_df[['danger_above_treeline',
                                                                                              'danger_below_treeline',
                                                                                              'danger_near_treeline']].applymap(convert_avy_str_to_num)



/Users/krex/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [98]:
type(ski_df['danger_above_treeline'][2456])

numpy.float64

In [99]:
ski_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21303 entries, 320 to 42119
Data columns (total 17 columns):
Date/Time (PST)             21303 non-null datetime64[ns]
Battery Voltage (v)         21296 non-null float64
Temperature (deg F)         21295 non-null float64
Wind Speed Minimum (mph)    21295 non-null float64
Wind Speed Average (mph)    21295 non-null float64
Wind Speed Maximum (mph)    21295 non-null float64
Wind Direction (deg.)       19429 non-null float64
date                        21303 non-null datetime64[ns]
24 Hour Snow (in)           17685 non-null float64
Total Snow Depth (in)       17685 non-null float64
Relative Humidity (%)       178 non-null float64
area                        19682 non-null object
danger_above_treeline       19346 non-null float64
danger_below_treeline       19346 non-null float64
danger_near_treeline        19346 non-null float64
date_tomorrow               19682 non-null object
month                       21303 non-null int64
dtypes: dateti

In [100]:
ski_df.tail(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,month
42119,2014-12-17 13:00:00,0.0,12.89,25.3,0.008,0.81,125.4,2014-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12


In [101]:
ski_df.describe() 

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),danger_above_treeline,danger_below_treeline,danger_near_treeline,month
count,21296.000000,21295.000000,21295.000000,21295.000000,21295.000000,19429.000000,17685.000000,17685.000000,178.000000,19346.000000,19346.000000,19346.000000,21303.000000
mean,12.543848,26.028648,2.420495,6.437304,12.053108,179.496993,0.018083,91.418860,70.643539,2.477618,1.954202,2.301458,4.368399
std,2.549749,78.319369,5.125817,6.052230,8.402773,99.822767,91.868543,29.605817,23.062710,0.780573,0.775944,0.771484,3.969149
min,0.000000,-7999.000000,-7.743000,0.000000,0.000000,0.000000,-7999.000000,-162.700000,20.510000,1.000000,1.000000,1.000000,1.000000
25%,12.610000,21.580000,0.000000,2.899000,7.212000,86.300000,0.283000,73.360000,52.417500,2.000000,1.000000,2.000000,2.000000
50%,12.660000,26.780000,0.285000,4.654000,10.000000,241.600000,0.512000,91.600000,71.955000,2.000000,2.000000,2.000000,3.000000
75%,12.740000,31.870000,2.877000,8.090000,14.250000,270.700000,2.425000,114.000000,91.775000,3.000000,2.000000,3.000000,4.000000
max,45.980000,68.510000,53.250000,84.900000,71.790000,359.700000,166.300000,220.500000,100.000000,4.000000,4.000000,4.000000,12.000000


In [ ]:
# to not loose the timestamp's hour, i have to but the date/time into a str before converting to json

In [102]:
ski_df = ski_df.reset_index(drop=True)

In [103]:
ski_df['Date/Time (PST)'] = ski_df['Date/Time (PST)'].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [104]:
ski_df.to_json('stevens_data_master.json')

In [106]:
pd.read_json('stevens_data_master.json').head()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,month
0,2019-04-30 23:00:00,12.80,33.07,5.962,9.230,13.33,267.0,2019-04-30,54.890,74.06,52.35,None,NaN,NaN,NaN,None,4
1,2019-04-30 22:00:00,12.82,34.52,4.910,11.970,16.22,275.2,2019-04-30,56.090,73.82,44.66,None,NaN,NaN,NaN,None,4
10,2019-04-30 13:00:00,12.67,41.23,0.000,4.946,18.17,292.8,2019-04-30,62.730,73.30,24.97,None,NaN,NaN,NaN,None,4
100,2019-04-26 19:00:00,12.69,34.47,1.052,15.430,41.23,296.0,2019-04-26,67.570,73.60,82.40,None,NaN,NaN,NaN,None,4
1000,2019-03-24 07:00:00,12.74,29.79,5.612,8.280,11.81,274.0,2019-03-24,0.491,90.00,NaN,Stevens Pass,1.0,1.0,1.0,"March 25, 2019",3
